## 1. Notebook: sloopvoorspelling.ipynb
### Doel en context
Het doel van dit notebook is om aan de hand van openbare data (zoals bouwjaar, woningtype, oppervlakte) te voorspellen of een gebouw sloopkandidaat is. Dit sluit aan bij een groter project waarbij circulair bouwen wordt gestimuleerd: het vroegtijdig detecteren van sloopprojecten kan bijdragen aan het hergebruik van materialen en efficiëntere ruimtelijke planning.

### Achtergrond van het probleem
Gebouwen worden vaak gesloopt omdat ze economisch verouderd zijn, niet meer voldoen aan moderne eisen of vanwege herontwikkeling van wijken. Door een machine learning-model te trainen op kenmerken van bestaande gebouwen (zoals bouwjaar en woningtype), wordt geprobeerd te voorspellen of een gebouw binnen deze risicocategorie valt.

## STAP 1. Dataset inladen

In [2]:
import pandas as pd
df_full = pd.read_csv("Heerlen_dataset.csv")

## STAP 2: Data voorbereiden

In [3]:
df = df_full[['bouwjaar', 'pandstatus', 'woningtype',
              'opp_adresseerbaarobject_m2', 'verhouding_opp_vbo_opp_pnd']].copy()

df['woningtype'] = df['woningtype'].fillna('Onbekend')

huidig_jaar = 2025
df['leeftijd'] = huidig_jaar - df['bouwjaar']

def inschat_levensduur(woningtype):
    if woningtype in ['Appartement', 'Tussenwoning', 'Hoekwoning', 'Vrijstaande woning', 'Woonhuis']:
        return 75
    elif woningtype in ['Kantoor', 'Winkel', 'Bedrijfspand']:
        return 50
    elif woningtype == 'Onbekend':
        return 60
    else:
        return 60

df['levensduur'] = df['woningtype'].apply(inschat_levensduur)
df['relatieve_ouderdom'] = df['leeftijd'] / df['levensduur']

df['pandstatus'] = df['pandstatus'].astype('category').cat.codes
df['woningtype'] = df['woningtype'].astype('category')
woningtype_categories = df['woningtype'].cat.categories
df['woningtype'] = df['woningtype'].cat.codes

df['sloopkans'] = (df['bouwjaar'] < 1970).astype(int)

### Data cleaning en preprocessing
De relevante data worden ingelezen uit Heerlen_dataset.csv. Er wordt gefilterd op bruikbare kolommen:  
`df = df_full[['bouwjaar', 'pandstatus', 'woningtype','opp_adresseerbaarobject_m2', 'verhouding_opp_vbo_opp_pnd']]`

### Waarom deze features?  

- `bouwjaar`: sleutelkenmerk – oudere gebouwen zijn vaker sloopkandidaten.

- `woningtype`: indicator voor structurele duurzaamheid en hergebruikpotentieel.

- `opp_adresseerbaarobject_m2`: totale oppervlakte zegt iets over schaal van pand.

- `verhouding_opp_vbo_opp_pnd`: geeft aan hoeveel van een pand werkelijk in gebruik is (efficiëntie).

### Omgaan met ontbrekende data
Lege woningtypes worden ingevuld als "Onbekend": 

`df['woningtype'] = df['woningtype'].fillna('Onbekend')`  

Zo blijven deze records bruikbaar en kan het model ook leren van panden met onvolledige informatie.



### Machine Learning model: RandomForestClassifier

### Label genereren voor training
In afwezigheid van echte sloopdata wordt een regelmatige inschatting gemaakt:


`df['sloopkans'] = (df['bouwjaar'] < 1970).astype(int)`  

Gebouwen ouder dan 1970 worden gelabeld als “risico op sloop”.

### Feature engineering: economische levensduur
Er wordt een belangrijke extra feature toegevoegd: de relatieve ouderdom ten opzichte van verwachte levensduur:

`df['relatieve_ouderdom'] = df['leeftijd'] / df['levensduur']`  

Dit bootst na hoe sloopbeslissingen in het echt gemaakt worden, en verhoogt de contextgevoeligheid van het model.

## STAP 3: Model trainen

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X = df[['bouwjaar', 'woningtype', 'verhouding_opp_vbo_opp_pnd',
        'opp_adresseerbaarobject_m2', 'relatieve_ouderdom']]
y = df['sloopkans']

model = RandomForestClassifier(random_state=42)
model.fit(X, y)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## STAP 4: Functie voor voorspelling op basis van cijfers

In [13]:
import numpy as np

def predict_sloop(bouwjaar, woningtype, verhouding_opp, opp_m2, relatieve_ouderdom):
    X_input = np.array([[bouwjaar, woningtype, verhouding_opp, opp_m2, relatieve_ouderdom]])
    pred = model.predict(X_input)[0]
    prob = model.predict_proba(X_input)[0][1]
    return pred, prob

### Modelopbouw en gebruik

`model = RandomForestClassifier(random_state=42)`
`model.fit(X, y)`  

Features:

- `bouwjaar`

- `woningtype (encoded)`

- `verhouding_opp`

- `opp_m2`

- `relatieve_ouderdom`

Het model wordt op deze vijf kenmerken getraind. Random Forest is gekozen vanwege:

- robuustheid bij heterogene data

- goede prestaties met categoriale en continue features

- uitlegbaarheid en betrouwbaarheid

## 5. Functie om adres op te zoeken en te voorspellen

In [14]:
def predict_sloop_by_address(df_full, straatnaam, huisnummer, huisletter=None):
    straatnaam = straatnaam.strip().lower()
    huisletter = huisletter.strip().upper() if huisletter else ''

    df_copy = df_full.copy()
    df_copy['openbareruimtenaam'] = df_copy['openbareruimtenaam'].astype(str).str.strip().str.lower()
    df_copy['huisletter'] = df_copy['huisletter'].fillna('').astype(str).str.strip().str.upper()

    query = (
        (df_copy['openbareruimtenaam'] == straatnaam) &
        (df_copy['huisnummer'] == huisnummer) &
        (df_copy['huisletter'] == huisletter)
    )

    matching = df_copy[query]
    if matching.empty:
        return f"❌ Geen match gevonden voor {straatnaam.title()} {huisnummer}{huisletter}."

    row = matching.iloc[0]

    try:
        bouwjaar = float(row['bouwjaar'])
        woningtype = row['woningtype']
        if pd.isna(woningtype):
            woningtype = 'Onbekend'
        woningtype = str(woningtype)

        verhouding_opp = float(row['verhouding_opp_vbo_opp_pnd'])
        opp_m2 = float(row['opp_adresseerbaarobject_m2'])

        leeftijd = 2025 - bouwjaar
        levensduur = inschat_levensduur(woningtype)
        relatieve_ouderdom = leeftijd / levensduur
    except Exception as e:
        return f"⚠️ Onvoldoende data om voorspelling te doen: {e}"

    try:
        woningtype_encoded = woningtype_categories.get_loc(woningtype)
    except KeyError:
        return f"⚠️ Woningtype '{woningtype}' zat niet in de trainingsdata."

    pred, prob = predict_sloop(bouwjaar, woningtype_encoded, verhouding_opp, opp_m2, relatieve_ouderdom)
    status = "✅ Kans op sloop" if pred == 1 else "🟩 Geen directe sloopindicatie"

    return {
        "adres": f"{straatnaam.title()} {huisnummer}{huisletter}",
        "bouwjaar": bouwjaar,
        "woningtype": woningtype,
        "leeftijd": leeftijd,
        "geschatte levensduur": levensduur,
        "relatieve ouderdom": round(relatieve_ouderdom, 2),
        "opp_m2": opp_m2,
        "verhouding_opp": verhouding_opp,
        "voorspelling": status,
        "sloopkans (model)": round(prob * 100, 2)
    }

## STAP 6: Terminal-functie om adres op te geven

In [15]:
def terminal_sloopcheck(df):
    print("📍 Geef een adres op om sloopkans te voorspellen:\n")

    straat = input("Straatnaam: ").strip()
    try:
        nummer = int(input("Huisnummer: ").strip())
    except ValueError:
        print("❌ Ongeldig huisnummer.")
        return

    letter_input = input("Huisletter (optioneel): ").strip()
    huisletter = letter_input.upper() if letter_input else None

    resultaat = predict_sloop_by_address(df, straat, nummer, huisletter)

    print("\n📊 Voorspelling resultaat:")
    if isinstance(resultaat, dict):
        for k, v in resultaat.items():
            print(f"{k}: {v}")
    else:
        print(resultaat)

### Voorspellen op adresniveau
Een gebruiker kan een adres invullen en zien wat de voorspelling is:

`resultaat = predict_sloop_by_address(df, 'Looierstraat', 15)`

Als woningtype ontbreekt of niet bekend was in de training, wordt dat duidelijk gemeld en alsnog ingevuld met `Onbekend`.

## STAP 7: Gebruik:

In [16]:
terminal_sloopcheck(df_full)

📍 Geef een adres op om sloopkans te voorspellen:


📊 Voorspelling resultaat:
adres: Gravenstraat 12
bouwjaar: 1930.0
woningtype: Tussen of geschakelde woning
leeftijd: 95.0
geschatte levensduur: 60
relatieve ouderdom: 1.58
opp_m2: 179.0
verhouding_opp: 2.18
voorspelling: ✅ Kans op sloop
sloopkans (model): 100.0


/opt/anaconda3/envs/zm001ds/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/zm001ds/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
